Preparing Simplified Street Graph
==============================
Start with this file.

It downloads the raw data from OSM and prepares a simplified street graph, consisting of one node for every intersection and one edge for every street segment. This simplified street graph is also referred to as *centerline graph*. The centerline graph also gets enriched with additional data sources such as public transit routes, elevation or traffic volumes. The resulting data is saved in the *process* folder and can be used by other scripts for further work.

In [ ]:
import copy, os
import snman
from snman.constants import *
from snman import osmnx_customized as oxc

#PERIMETER = 'birchplatz'
PERIMETER = 'zrh_north-west'
#PERIMETER = 'matsim_zrh5'
#PERIMETER = 'bucheggplatz'
#PERIMETER = 'hardbruecke'
#PERIMETER = 'aarau_buchs_suhr_5km'
#PERIMETER = 'aarau_buchs_suhr_10km'
#PERIMETER = 'ebc_zrh_v01_10km'
#PERIMETER = 'hb'
#PERIMETER = 'cambridge_extended'
#PERIMETER = 'chicago_logan_sq'
#PERIMETER = 'cambridge'
#PERIMETER = 'zurich'

# set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', '_debug')
#process_path = os.path.join(data_directory, 'process', PERIMETER)

#CRS_internal = 29119    # for Boston
#CRS_internal = 32216    # for Chicago
CRS_internal = 2056      # for Zurich
CRS_for_export = 4326
oxc.settings.useful_tags_way = OSM_TAGS

Loading data
------------
Loads the necessary datasets, including downloading the raw OSM data

In [ ]:
print('Load perimeters')
perimeters = snman.io.load_perimeters(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'),
    crs=CRS_internal
)

print('Get data from OSM server')
# At this step, simplification means only removing degree=2 edges
G_raw = oxc.graph_from_polygon(
    # set the perimeter here
    perimeters.to_crs(4326).loc[PERIMETER]['geometry'],
    custom_filter=snman.constants.OSM_FILTER,
    simplify=True, simplify_strict=False, retain_all=True, one_edge_per_direction=False
)

if 1:
    print('Export raw street graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.io.export_street_graph(
        G_raw,
        os.path.join(process_path, 'raw_street_graph_edges.gpkg'),
        os.path.join(process_path, 'raw_street_graph_nodes.gpkg'),
        crs=CRS_for_export
    )

G = copy.deepcopy(G_raw)

print('Load manual intersections')
# polygons used to override the automatically detected intersections in some situations
given_intersections_gdf = snman.io.load_intersections(
    os.path.join(inputs_path, 'intersection_polygons/intersection_polygons.shp'),
    crs=CRS_internal
)

print('Prepare graph')
snman.street_graph.prepare_graph(G)

print('Convert CRS of street graph')
snman.street_graph.convert_crs(G, CRS_internal)

print('Generate lanes')
# interpreting the OSM tags into a collection of lanes on each edge
snman.space_allocation.generate_lanes(G)

print('Filter graph by modes')
G = snman.street_graph.filter_lanes_by_modes(G, {MODE_TRANSIT, MODE_PRIVATE_CARS, MODE_CYCLING, MODE_CAR_PARKING})

print('Load sensors')
sensors_df = snman.io.load_sensors(os.path.join(inputs_path, 'sensors/sensors.csv'))

print('Load public transit routes')
pt_routes = snman.io.load_public_transit_routes(
    #os.path.join(inputs_path, "switzerland/aarau/public_transit/avk_oevlinien_20221211.shp)",
    os.path.join(inputs_path, "switzerland/zurich/public_transit/ZVV_LINIEN_GEN_L.shp"),
    perimeter=perimeters.loc[PERIMETER]['geometry']
)
# keep only bus and tram lines (no micro buses and night buses)
pt_routes = pt_routes.query("TYPE in ['Bus', 'Tram'] and ALIGNMENT != 'tunnel'")

print('Load parking spots')
parking_spots = snman.io.load_parking_spots(os.path.join(inputs_path, 'switzerland/zurich/strassenparkplaetze.gpkg'), crs=CRS_internal)

Enriching the street graph before simplification
------------------------------------------------
Enrichment steps that need to be done before simplification

In [ ]:
if 1:
    print('Add sensors')
    snman.enrichment.match_sensors(G, sensors_df)

if 1:
    print('Identify hierarchy')
    # split the edges into hierarchy categories, such as main roads, local roads, etc.
    snman.hierarchy.add_hierarchy(G)
    
if 0:
    print('Add parking spaces')
    snman.enrichment.match_parking_spots(G, parking_spots)

Simplification
--------------
Consolidates intersections and merges edges so that we obtain a centerline graph. The process needs to be repeated a few times to catch all secondary simplification possibilities

In [ ]:
if 1:
    print('Light simplification of edge geometries')
    snman.simplification.simplify_edge_geometries(G, radius=2)

for i in range(4):

    print('ITERATION', i)
    
    # here, you can choose whether some roads should be excluded from the simplification
    def add_include_in_simplification_to_edge(G, uvk):
        data = G.edges[uvk]
        #data['_include_in_simplification'] = data.get('hierarchy') != snman.hierarchy.HIGHWAY
        data['_include_in_simplification'] = True
    snman.graph.apply_function_to_each_edge(G, add_include_in_simplification_to_edge)
    
    def add_include_in_simplification_to_node(G, n):
        uvks = set(G.in_edges(n, keys=True)).union(set(G.out_edges(n, keys=True)))
        edges_included_in_simplification = [G.edges[uvk].get('_include_in_simplification') for uvk in uvks]
        G.nodes[n]['_include_in_simplification'] = False not in edges_included_in_simplification
        #G.nodes[n]['_include_in_simplification'] = True
    snman.graph.apply_function_to_each_node(G, add_include_in_simplification_to_node)

    print('Detect intersections')
    snman.simplification.add_layers_to_nodes(G)
    intersections_gdf = snman.simplification.merge_nodes_geometric(
        G,
        given_intersections_gdf=given_intersections_gdf
    )

    print('Split through edges in intersections')
    snman.simplification.split_through_edges_in_intersections(G, intersections_gdf)

    print('Detect intersections (repeat to ensure that no points are outside of intersections)')
    snman.simplification.add_layers_to_nodes(G)
    intersections_gdf = snman.simplification.merge_nodes_geometric(
        G,
        given_intersections_gdf=given_intersections_gdf
    )
    
    print('Add layers and hierarchies to nodes')
    snman.simplification.add_layers_to_nodes(G)
    snman.graph.apply_function_to_each_node(G, snman.street_graph_node.add_hierarchies)

    print('Add connections between components in intersections')
    snman.simplification.connect_components_in_intersections(G, intersections_gdf, separate_layers=True)

    print('Consolidate intersections')
    G = snman.simplification.consolidate_intersections(
        G, intersections_gdf,
        reconnect_edges=True
    )

    print('Merge consecutive edges')
    snman.simplification.add_layers_to_nodes(G)
    snman.merge_edges.merge_consecutive_edges(G)

    print('Merge parallel edges')
    snman.merge_edges.merge_parallel_edges(G)

    print('Update precalculated attributes')
    snman.street_graph.update_precalculated_attributes(G)

print('Heavy simplification of edge geometries')
#snman.simplification.simplify_edge_geometries(G)

print('Keep only the largest connected component')
G = snman.graph.keep_only_the_largest_connected_component(G, weak=True)

Updating pre-calculated attributes
----------------------------------
Updates the OSM tags and stats like aggregate lane widths to match the simplified graph

In [ ]:
print('Add lane stats to edges')
# how many lanes, how wide, etc.
snman.space_allocation.generate_lane_stats(G)

print('Update OSM tags')
# to match the simplified and merged edges
snman.space_allocation.update_osm_tags(G)

if 0:
    print('Update street counts per node')
    spn = oxc.stats.count_streets_per_node(G, nodes=G.nodes)
    nx.set_node_attributes(G, values=spn, name="street_count")

Enrichment
----------
Add additional data to the centerline graph, such as public transit, elevation, and traffic counts

In [ ]:
if 0:
    print('Add public transit')
    snman.enrichment.match_public_transit(G, pt_routes)

if 1:
    print('Add elevation')
    G = oxc.elevation.add_node_elevations_raster(
        G,
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'ch_dhm_25', 'ch_dhm_2056.tif'),
        #raster_crs=2056,
        os.path.join(inputs_path, 'copernicus_dem', 'Copernicus_DSM_10_N41_00_W088_00', 'DEM', 'Copernicus_DSM_10_N41_00_W088_00_DEM.tif'),
        raster_crs=4326, #Chicago
        graph_crs=CRS_internal,
        cpus=1
    )
    G = oxc.elevation.add_edge_grades(G, add_absolute=False)

if 0:
    import geopandas as gpd
    print('Add traffic counts')
    source = gpd.read_file(os.path.join(inputs_path, 'traffic_volumes/npvm_2017_filtered.gpkg')).to_crs(2056)
    source['fid'] = source.index
    # Remove links with zero traffic (otherwise they will distort the averages on the matched links)
    source = source[source['DTV_ALLE'] > 0]
    snman.enrichment.match_linestrings(G, source, [
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_avg',         'agg': 'avg' },
        {'source_column': 'DTV_ALLE',   'target_column': 'adt_max',         'agg': 'max' },
        {'source_column': 'FROMNODENO', 'target_column': 'npvm_fromnodeno', 'agg': 'list'},
        {'source_column': 'TONODENO',   'target_column': 'npvm_tonodeno',   'agg': 'list'}
    ])
if 0:
    snman.street_graph.add_edge_costs(G)

In [ ]:
if 0:
    snman.space_allocation.normalize_cycling_lanes(G)

Export
------
Save the datasets to the hard drive. All files can be opened in QGIS using the *snman_detailed.qgz* file.

In [ ]:
if 1:
    print('Export street graph')
    # each street is one edge, the lanes are saved as an attribute
    snman.io.export_street_graph(
        G,
        os.path.join(process_path, 'street_graph_edges.gpkg'),
        os.path.join(process_path, 'street_graph_nodes.gpkg'),
        crs=CRS_for_export
    )

if 1:
    print('Export lane geometries')
    # each lane has an own geometry and with as an attribute, for visualization purposes
    snman.io.export_street_graph_with_lanes(
        G,
        KEY_LANES_DESCRIPTION,
        os.path.join(process_path, 'lane_geometries.shp'),
        scaling=1, crs=CRS_for_export
    )

if 1:
    print('Save intersection geometries into a file')
    snman.io.export_gdf(
        intersections_gdf,
        os.path.join(process_path, 'intersections_polygons.gpkg'),
        columns=['geometry'], crs=CRS_for_export
    )

In [ ]:
if 0:
    #TODO: OSM Export breaks the street centerline graph when using the boston projection
    print('Export as OSM')
    snman.io.export_osm_xml(
        G,
        os.path.join(process_path, 'before.osm'),
        EXPORT_OSM_TAGS,
        uv_tags=True, as_oneway_links=False, key_lanes_description=KEY_LANES_DESCRIPTION
    )
    snman.io.export_osm_xml(
        G,
        os.path.join(process_path, 'before_oneway_links.osm'),
        EXPORT_OSM_TAGS,
        uv_tags=True, as_oneway_links=True, key_lanes_description=KEY_LANES_DESCRIPTION
    )